# Replicating Curtis (2012, TAR): A Fundamental-Analysis-Based Test for Speculative Prices


**Ohlson’s Residual Income Model (RIM) - Theoretical Basis**

Ohlson’s RIM expresses firm value $ V_t $ as:
$$ V_t = B_t + \sum_{i=1}^{\infty} \frac{E_t[RI_{t+i}]}{(1 + r)^i} $$

Where:
-  $B_t$ = Book value of equity at time \( t \) (in the code: `ceq`).
- $RI_{t+i}$ = Residual income at time \( t+i \), defined as $ NI_{t+i} - r \cdot B_{t+i-1} $, where $ NI_{t+i} $ is net income (in the code: `ib`), and $ r$ is the cost of equity capital.
- $ E_t[\cdot] $ = Expectation operator based on information at \( t \).
- $r$ = Discount rate (cost of equity).

For practical estimation, Ohlson assumes a perpetuity or simplified growth model. In Curtis (2012), the fundamental value $Vf(x)$ is calculated assuming residual income persists indefinitely with a constant growth rate $g$ , leading to:
$$ Vf(x)_t = B_t + \frac{RI_{t+1}}{r - g} $$

Where:
- $ RI_{t+1} = NI_{t+1} - r \cdot B_t $
- $ g $ = Long-term growth rate of residual income.

This perpetuity assumption simplifies the infinite sum into a closed-form expression, commonly used in empirical applications of RIM.

## Step 1: Data Preparation

### 1.0. Setting up SAS ENV

In [1]:
* Libnames and Options                                               ;
libname crsp ("/wrds/crsp/20250403/a_ccm" "/wrds/crsp/20250403/a_stock" "/wrds/crsp/20250403/a_indexes"); 
libname ccm ("/wrds/crsp/20250403/a_ccm" ); 
libname comp ("/wrds/comp/20250403/d_na"); 
libname ff "/wrds/ff/20250403";
libname out '/sastemp/'; /* Location for output datasets */

options symbolgen;
options mprint mlogic;
*options fmtsearch=(out); /* For custom formats if needed */

SAS Connection established. Subprocess id is 3862078


21   ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
22   
23   * Libnames and Options                                               ;
24   libname crsp ("/wrds/crsp/20250403/a_ccm" "/wrds/crsp/20250403/a_stock" "/wrds/crsp/20250403/a_indexes");
NOTE: Libref CRSP was successfully assigned as follows: 
      Levels:           3
      Engine(1):        V9 
      Physical Name(1): /wrds/crsp/20250403/a_ccm
      Engine(2):        V9 
      Physical Name(2): /wrds/crsp/20250403/a_stock
      Engine(3):        V9 
      Physical Name(3): /wrds/crsp/20250403/a_indexes
25   libname ccm ("/wrds/crsp/20250403/a_ccm" "/wrds/crsp/20250403/a_stock" "/wrds/crsp/20250403/a_indexes");
NOTE: Libref CCM was successfully assigned as follows: 
      Levels:           3
      Engine(1):        V9 


In [2]:
* --- Macro Variables --- *;
%let start_year = 1979;
%let end_year = 2024;
%let start_date = "01JAN&start_year"d;
%let end_date = "31DEC&end_year"d;
%let comp_start_year = %eval(&start_year - 2); /* Go back further for lags */
%let comp_end_year = &end_year;
%let equity_premium = 0.06; /* 6% equity premium */
%let growth_rate = 0.03;    /* 3% perpetual growth rate (g) */
%let report_lag_mths = 4;   /* Months after FYE until data is assumed available */


36   ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
37   
38   * --- Macro Variables --- *;
39   %let start_year = 1979;
40   %let end_year = 2024;
SYMBOLGEN:  Macro variable START_YEAR resolves to 1979
41   %let start_date = "01JAN&start_year"d;
SYMBOLGEN:  Macro variable END_YEAR resolves to 2024
42   %let end_date = "31DEC&end_year"d;
SYMBOLGEN:  Macro variable START_YEAR resolves to 1979
43   %let comp_start_year = %eval(&start_year - 2); /* Go back further for lags */
SYMBOLGEN:  Macro variable END_YEAR resolves to 2024
44   %let comp_end_year = &end_year;
45   %let equity_premium = 0.06; /* 6% equity premium */
46   %let growth_rate = 0.03;    /* 3% perpetual growth rate (g) */
47   %let report_lag_mths = 4;   /* Months after FYE until data is assumed available */
48   
49   ods html5 (id=saspy_internal) close;ods listing;



### Useful WRDS SAS MACROS

[Useful WRDS SAS MACROS at https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/)

#### Macros

WRDS Research Macros

* [Betas (CIZ Format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-betas-ciz/ "Compute Market-Model Beta for all CRSP securities on a rolling daily or monthly basis.")
* [Betas \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-betas/ "Compute Market-Model Betas using CRSP data.")
* [CCM \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-ccm/ "Use CRSP-Compustat Merged Table to Add Permno to Compustat Data")
* [ccm\_lnktable.sas \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-ccm_lnktablesas/ "This program shows the main processing part of the CCM web queries.")
* [COMPOUND (CIZ format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-compound-ciz/ "The Compound macro calculates continuously compounded returns with an option to account for delisting returns. It uses returns in input file (INSET) provided at a given frequency (INFREQ) over the time intervals specified by OUTFREQ (annual, semi-annual, quarterly, monthly or weekly). It also provides maximum and minimum returns, number of total and missing observations into the output dataset OUTSET.")
* [COMPOUND \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-compound/ "Calculates continuously compounded returns at the frequency requested by the user.")
* [CRSPMERGE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-crspmerge/ "Merges CRSP Stocks and Events Data")
* [CSV \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-marcos-csv/ "Exports a SAS dataset Into an Excel or CSV file.")
* [cvccmlnk.sas \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-cvccmlnksas/ "This macro creates the CCMXPF_LINKDTABLE data set. It contains data from the Link History table as well as the USEDFLAG variable from the Link Used table.")
* [EVTSTUDY \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-evtstudy/ "Performs an event study")
* [FFI48 \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-ffi48/ "Creates Fama & French 48 Industry Classification Variable")
* [FM \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-fm/ "Performs Fama-MacBeth Regressions. Calculates FM coefficients with Newey-West adjusted standard errors")
* [ICLINK (CIZ format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-iclink-ciz/ "ICLINK creates a link table between IBES TICKER and CRSP PERMNO. Scores link from 0 (best link) to 6.")
* [ICLINK \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-iclink/ "ICLINK creates a link table between IBES TICKER and CRSP PERMNO . Scores links from 0 (best link) to 6.")
* [IDVOL (CIZ format)\- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-idvol-ciz/ "Calculates idiosyncratic volatility using time-series monthly/daily regressions for various risk models")
* [IDVOL \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-idvol/ "Calculates idiosyncratic volatility using time-series monthly/daily regressions for various risk models")

* [INDCLASS \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-indclass/ "Constructs 4 different industry classifications based on SIC, NAICS, GICS and Fama-French industry classifications")
* [INDRATIOS (CIZ format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-indratios-ciz/ "Computes a broad range of financial ratios aggregated at the industry level using Fama-French industry classification")
* [INDRATIOS \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-indratios/ "Computes a broad range of financial ratios aggregated at the industry level using Fama-French industry classification")
* [LINEPARAPARSE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-lineparaparse/ "WRDS-SEC Paragraph Parser, Around a Predefined Line Number")
* [Macro \- TEXTPARSE](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-textparse/ "WRDS-SEC Filings Text Parser")
* [MAKE DUMMIES \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-make-dummies/ "MAKE_DUMMIES")
* [Market to Book Ratios \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-market-book-ratios/ "Calculate Raw and Industry Adjusted Market-to-Book Ratio using separately Compustat only and CRSP-Compustat Merged")
* [merge\_funda\_crsp\_bycusip.sas \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-merge_funda_crsp_bycusipsas/ "This Program merges CRSP and Compusat Xpressfeed databases by CUSIP. To be able to run the program, a user should have access to Compustat Annual Xpressfeed datasets and CRSP monthly database")
* [Momentum Strategies/Portfolios \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-momentum-strategiesportfolios/ "Replicates Jegadeesh and Titman (JF, 1993) Momentum Portfolios")
* [NEUT \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-neut/ "Neutralizes (orthogonalizes) a set of variables by a list of numerical or categorical factors")
* [NWORDS \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-nwords/ "Counts the Number of Words Within a Text String")
* [OCLINK \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-oclink/ "Creates OptionMetrics-CRSP Link Table")
* [Option Pricing Models \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-option-pricing-models/ "Binomial Option Pricing Model for American Options")
* [PARAPARSE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-paraparse/ "WRDS-SEC Paragraph Parser, from a text string")
* [POPULATE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-populate/ "Populates a Dataset with Any Frequency Into Monthly Intervals")
* [Portfolios by Size (CIZ format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/macros-portfolios-size-ciz/ "Size Portfolios for all CRSP Securities")

* [Portfolios by Size \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/macros-portfolios-size/ "Size Portfolios for all CRSP Securities")
* [QUARTERIZE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-quarterize/ "Quarterizes Compustat YTM Cash Flow Variables in FUNDQ Dataset")
* [Return Gap \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-return-gap/ "This program is intended to replicate some of the results of the paper \"Unobserved Actions of Mutual Funds\" by Kacperzczyk, Sialm and Zheng (RFS, 2008). The authors developed the “Return Gap” measure as a means to capture the impact of unobserved actions by mutual fund managers on fund returns.")
* [RRLOOP \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/rrloop-rolling-regresson-macro/ "With this rolling regression macro, the least-squares equation is estimated multiple times using partially overlapping subsamples from a larger set.")
* [Run an Event Study (CIZ Format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/run-an-event-study-ciz-format-macro/ "The Event Study Macro calculates Cumulative Abnormal Returns and various t-statistics (Patells t, Sign t, etc) for an input dataset (Inset) containing (permno,evtdate) combinations. Outputs mean cumulative abnormal return within the event window (set by the variables start and end) and different statistics into the output dataset (Outset). Abnormal returns are computed using either Market, Fama-French or Carhart 4-factor models during the estimation period. The program draws on some of the ideas outlined in the event study macro in Chapter 6 of Boehmer, Broussard and Kallunki (2002)")
* [Run an Event Study \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-run-event-study/ "Performs an event study with SAS macro.")
* [SIZE\_BM \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-size_bm/ "WRDS SAS Macro for creating Fama-French Size and Book-to-Market 2 by 3 portfolios using CRSP, Compustat, and CCM")
* [taq6\.sas \- Program](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-program-taq6sas/ "Retrieve Transaction price for a fixed interval, e.g., every 15 minutes.")
* [taq\_daily\_variables.sas \- Program](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-program-taq_daily_variablessas/ "An example program for taq_daily_variables.sas")
* [TAQ\_EVENT\_WINDOWS \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/taq_event_windows/ "Retrieves trade or quote data from TAQ for user-specified SYMBOLS, each with a unique date and time range.")
* [TCLINK \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-tclink/ "Create TAQ-CRSP Link Table")
* [Trace \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-trace/ "Find a last daily TRACE record for each CUSIP")
* [TRADE\_DATE\_WINDOWS (CIZ format) \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-trade_date_windows-ciz/ "Using a trading date calendar file (either daily or monthly), create a dataset of trading date \"windows\". The dataset will contain one record per window, with three variables: BEG_DATE, END_DATE, and WINDOW_SIZE. All windows will be entirely within the range of the calendar file. Non-trading dates are ignored.")
* [TRADE\_DATE\_WINDOWS \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-trade_date_windows/ "Using a trading date calendar file (either daily or monthly), create a dataset of trading date \"windows\". dataset will contain one record per window, with three variables: BEG_DATE, END_DATE,and WINDOW_SIZE). All windows will be entirely within the range of the calendar file.")
* [VW\_AVGPRICE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-vw_avgprice/ "VW_AVGPRICE")
* [WINSORIZE \- Macro](https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macros-winsorize/ "Winsorizes or Trims Outliers")


### 1.1. Compustat


In [3]:

data comp_annual;
    set comp.funda(where=(
     indfmt = 'INDL' /* Industrial format */
      and datafmt = 'STD' /* Standardized data format */
      and popsrc = 'D'  /* Domestic */
      and consol = 'C'  /* Consolidated */
      and curcd = 'USD' /* US Dollar */
      and fyear between &comp_start_year and &comp_end_year));
    keep gvkey datadate fyear sich fyr ceq ib dvc curcd at spi dvp tstkp dvpa sale rect invt aco ap lco dp ppent ppegt intan ao lo dltis dltr dlcch sstk prstkc dvc csho ajex;
run;

proc sort data=comp_annual;
    by gvkey datadate;
run;


51   ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
52   
53   
54   data comp_annual;
55       set comp.funda(where=(
56        indfmt = 'INDL' /* Industrial format */
57         and datafmt = 'STD' /* Standardized data format */
58         and popsrc = 'D'  /* Domestic */

3 The SAS System                                                                                                                                         20:42 Sunday, April 6, 2025

59         and consol = 'C'  /* Consolidated */
60         and curcd = 'USD' /* US Dollar */
SYMBOLGEN:  Macro variable COMP_START_YEAR resolves to 1977
61         and fyear between &comp_start_year and &comp_end_year));
SYMBOLGEN:  Macro variable COMP_END_YEAR resolves to 2024
NOTE: Data file COMP.FUNDA.DATA is in a format that is native to another host, or the file encoding does 

In [4]:
proc means data=comp_annual; run;

Variable,Label,N,Mean,Std Dev,Minimum,Maximum
datadate fyear ajex fyr aco ao ap at ceq csho dlcch dltis dltr dp dvc dvp dvpa ib intan invt lco lo ppegt ppent prstkc rect sale spi sstk tstkp sich,"Data Date Data Year - Fiscal Adjustment Factor (Company) - Cumulative by Ex-Date Fiscal Year-end Month Current Assets - Other - Total Assets - Other Accounts Payable - Trade Assets - Total Common/Ordinary Equity - Total Common Shares Outstanding Current Debt - Changes Long-Term Debt - Issuance Long-Term Debt - Reduction Depreciation and Amortization Dividends Common/Ordinary Dividends - Preferred/Preference Preferred Dividends in Arrears Income Before Extraordinary Items Intangible Assets - Total Inventories - Total Current Liabilities - Other - Total Liabilities - Other - Total Property, Plant and Equipment - Total (Gross) Property, Plant and Equipment - Total (Net) Purchase of Common and Preferred Stock Receivables - Total Sales/Turnover (Net) Special Items Sale of Common and Preferred Stock Treasury Stock - Preferrred Standard Industrial Classification - Historical",442532 442532 442522 442532 337922 365876 362927 366742 365964 414571 177990 339312 340924 350632 361566 364741 321531 365137 336815 359794 337875 366249 325921 359053 331336 358642 364508 351096 343547 333736 287176,15517.87 2001.53 29.3073389 10.0354957 93.9829781 736.3765279 2073.13 7364.87 1215.71 103.9735429 -7.1358038 625.2934645 578.8767856 126.7390869 65.7339935 3.2410838 0.1151499 134.3563090 586.6081729 332.4046087 289.7043713 1485.93 1987.67 1044.28 61.3522053 2410.41 2181.36 -23.5835990 37.9936846 0.0440235 4819.15,4845.70 13.2535913 3846.36 3.2673490 1080.51 13278.77 34632.18 76215.49 7610.73 2343.19 1731.28 25218.90 25857.56 794.7750913 497.9484266 198.3186184 3.2130289 1287.43 4479.50 5986.86 2101.53 23755.46 12660.76 6403.02 772.6967010 39164.54 12162.44 474.0154252 477.9832593 3.1783263 2045.79,6390.00 1977.00 1E-8 1.0000000 -56.9000000 -11139.00 -2.4570000 0 -139965.00 0 -189419.00 -770.0000000 -593.5690000 -112.0000000 -10.0000000 -1239.00 0 -99289.00 -40.4550000 -2.6830000 -89.0000000 -750.0000000 0 0 -372.2000000 -0.1670000 -24954.68 -51066.20 -516.7130000 -12.0000000 100.0000000,23831.00 2024.00 545454.55 12.0000000 197302.72 1630544.78 2462303.00 4349731.00 649368.00 606407.69 167482.47 6019062.22 6007354.94 52892.00 100117.00 85419.00 652.6000000 104821.00 310197.00 530917.00 382408.76 2095002.97 641121.77 328806.00 100390.00 4150750.00 680985.00 120517.00 77490.00 422.0000000 9998.00


### 1.2 CCM Macro

In [5]:
/* ********************************************************************************* */
/* ******************** W R D S   R E S E A R C H   M A C R O S ******************** */
/* ********************************************************************************* */
/* WRDS Macro: CCM                                                                   */
/* Summary   : Use CRSP-Compustat Merged Table to Add Permno to Compustat Data       */
/* Date      : October 20, 2010                                                      */
/* Author    : Luis Palacios and Rabih Moussawi, WRDS                                */
/* https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-ccm/     */
/* Variables : - INSET   : Input dataset: should have a gvkey and a date variable    */
/*             - DATEVAR : Date variable to be used in the linking                   */
/*             - LINKTYPE: List of Linktypes: LU LC LX LD LN LS NP NR NU             */
/*             - REMDUPS : Flag 0/1 to remove multiple secondary permno matches      */
/*             - OVERLAP : Date Condition Overlap, in years                          */
/*             - OUTSET  : Compustat-CRSP link table output dataset                  */
/* ********************************************************************************* */
 
%MACRO CCM (INSET=,DATEVAR=DATADATE,OUTSET=CCM,LINKTYPE=LULC,REMDUPS=1,OVERLAP=0);
 
/* Check Validity of CCM Library Assignment */
%if (%sysfunc(libref(CCM))) %then %do; libname CCM ("/wrds/crsp/sasdata/q_ccm/"); %end;
%if (%sysfunc(libref(CCM))) %then %do; libname CCM ("/wrds/crsp/sasdata/a_ccm/") ; %end;
%put; %put ### START. ;
 
/* Convert the overlap distance into months */
%let overlap=%sysevalf(12*&overlap.);
 
options nonotes;
/* Make sure first that the input dataset has no duplicates by GVKEY-&DATEVAR */
proc sort data=&INSET out=_ccm0 nodupkey; by GVKEY &DATEVAR; run;
 
/* Add Permno to Compustat sample */
proc sql;
create table _ccm1 as
select distinct b.lpermno as PERMNO " ", a.*, b.linkprim, b.linkdt
from _ccm0 as a, crsp.ccmxpf_lnkhist as b
where a.gvkey=b.gvkey and index("&linktype.",strip(b.linktype))>0
and (a.&datevar>= intnx("month",b.linkdt   ,-&overlap.,"b") or missing(b.linkdt)   )
and (a.&datevar<= intnx("month",b.linkenddt, &overlap.,"e") or missing(b.linkenddt));
quit;
  
/* Cleaning Compustat Data for no relevant duplicates                       */
/* 1. Eliminating overlapping matching : few cases where different gvkeys   */
/*   for same permno-date --- some of them are not 'primary' matches in CCM.*/
/*   Use linkprim='P' for selecting just one gvkey-permno-date combination; */
proc sort data=_ccm1;
  by &datevar permno descending linkprim descending linkdt gvkey;
run;
 
/* it ties in the linkprim, then use most recent link or keep all */
data _ccm2;
set _ccm1;
by &datevar permno descending linkprim descending linkdt gvkey;
if first.permno;
%if &REMDUPS=0 %then %do; drop linkprim linkdt; %end;
run;
  
%if &REMDUPS=1 %then
 %do;
   proc sort data=_ccm2; by &datevar gvkey descending linkprim descending linkdt;
   data _ccm2;
   set _ccm2;
   by &datevar gvkey descending linkprim descending linkdt;
   if first.gvkey;
   drop linkprim linkdt;
   run;
   %put ## Removed Multiple PERMNO Matches per GVKEY ;
 %end;
 
/* Sanity Check -- No Duplicates -- and Save Output Dataset */
proc sort data=_ccm2 out=&OUTSET nodupkey; by gvkey &datevar permno; run;
%put ## &OUTSET Linked Table Created;
 
/* House Cleaning */
proc sql;
 drop table _ccm0, _ccm1, _ccm2;
quit;
 
%put ### DONE . ; %put ;
options notes;
%MEND CCM;
 
 
/* ********************************************************************************* */
/* *************  Material Copyright Wharton Research Data Services  *************** */
/* ****************************** All Rights Reserved ****************************** */
/* ********************************************************************************* */


77   ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
78   
79   /* ********************************************************************************* */
80   /* ******************** W R D S   R E S E A R C H   M A C R O S ******************** */
81   /* ********************************************************************************* */
82   /* WRDS Macro: CCM                                                                   */
83   /* Summary   : Use CRSP-Compustat Merged Table to Add Permno to Compustat Data       */
84   /* Date      : October 20, 2010                                                      */
85   /* Author    : Luis Palacios and Rabih Moussawi, WRDS                                */
86   /* https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-ccm/     */
87   /* Variables : - INSET   : Input da

In [6]:
%CCM(INSET=comp_annual,DATEVAR=DATADATE,OUTSET=ccm_annual,LINKTYPE=LULC,REMDUPS=1,OVERLAP=0);
proc print data=ccm_annual(obs=30);run;

Obs,PERMNO,gvkey,datadate,fyear,ajex,curcd,fyr,aco,ao,ap,at,ceq,csho,dlcch,dltis,dltr,dp,dvc,dvp,dvpa,ib,intan,invt,lco,lo,ppegt,ppent,prstkc,rect,sale,spi,sstk,tstkp,sich
1,25881,001000,19771231,1977,1.000000000000,USD,12,0.6820,0.4290,4.0120,44.0250,15.8760,2.2260,.,10.6780,7.4000,2.5070,0.2220,0.0940,0.0000,1.9280,0.0000,9.1290,3.6910,0.0000,27.3540,19.9170,1.6380,12.4340,77.9460,0.9110,0.1330,.,.
2,10015,001001,19831231,1983,1.000000000000,USD,12,0.1220,0.0340,0.6500,14.0800,7.8230,3.5680,.,1.0040,1.4450,0.9530,0.0000,0.0000,0.0000,1.1350,0.6120,0.3230,0.7060,0.0000,12.4330,8.5360,0.0000,0.0820,25.3950,0.0000,5.7670,.,.
3,10015,001001,19841231,1984,1.000000000000,USD,12,0.2450,0.0540,1.3440,16.2670,8.9620,3.5680,-0.0780,0.2490,0.4120,1.3160,0.0000,0.0000,0.0000,1.1380,0.6310,0.5020,0.7880,0.0000,17.8310,12.7150,0.0000,0.0560,32.0070,0.0000,0.0000,.,.
4,10015,001001,19851231,1985,1.000000000000,USD,12,0.2970,0.1280,2.1930,39.4950,13.0140,3.9880,-7.7400,5.7470,3.1730,2.0090,0.0000,0.0000,0.0000,2.5760,13.0770,0.7390,2.1370,0.0000,29.0600,22.4110,0.0090,0.0290,53.7980,0.5890,1.4850,.,.
5,10031,001003,19831231,1983,1.000000000000,USD,12,0.1520,0.0000,0.4430,8.5290,6.0950,2.6830,.,0.0000,0.0530,0.0490,0.0000,0.0000,0.0000,1.0500,0.0000,5.4080,0.6950,0.0000,0.3760,0.2560,1.2000,0.6900,13.7930,0.0000,3.0620,0.0000,.
6,10031,001003,19841231,1984,1.000000000000,USD,12,0.1220,0.0000,0.3460,8.2410,6.4820,2.6830,0.0000,0.0000,0.2500,0.0930,0.0000,0.0000,0.0000,0.3870,0.0000,5.6610,0.4630,0.0000,0.6170,0.4240,0.0000,1.1900,13.8290,0.0000,0.0000,0.0000,.
7,10031,001003,19860131,1985,1.000000000000,USD,1,0.2720,0.6390,1.0930,13.9900,6.6650,2.6830,0.0000,4.6700,0.6190,0.3790,0.0000,0.0000,0.0000,0.2360,.,11.0710,1.5320,0.0000,1.3260,0.8930,0.0000,1.1100,24.1890,0.0000,0.0000,0.0000,.
8,10031,001003,19870131,1986,1.000000000000,USD,1,0.5140,0.4600,1.2700,14.5860,7.4580,2.6830,.,0.0000,0.9380,0.5290,0.0000,0.0000,0.0000,0.7930,0.0000,11.5630,1.8040,0.0000,1.8080,1.0360,0.0000,0.7720,36.3080,0.0000,0.0000,0.0000,.
9,10031,001003,19880131,1987,1.000000000000,USD,1,0.5360,0.6640,0.7130,16.0420,7.6430,2.6830,.,2.1470,0.4030,0.5160,0.0000,0.0000,0.0000,-0.5250,0.0000,11.9220,2.1780,0.0000,2.2970,1.1790,0.0000,1.2660,37.3560,0.0000,0.0000,0.0000,5712
10,10031,001003,19890131,1988,1.000000000000,USD,1,0.5260,0.5480,3.0110,16.2800,-0.1940,2.6830,.,2.2400,0.0170,0.6920,0.0000,0.0000,0.0000,-7.8380,0.0000,11.6800,5.7330,0.0000,2.5930,1.0760,0.0000,2.1480,32.8080,-3.5000,0.0000,0.0000,5712


### 1.3 Converting Compustat annual to monthly frequency

In [16]:

* Calculate availability dates for Compustat data *;
proc sort data=ccm_annual; by gvkey datadate; run;

data comp_monthly_avail;
    set ccm_annual;
    by gvkey;

    * Calculate the date when the annual data is assumed available (end of month) *;
    avail_date = intnx('MONTH', datadate, &report_lag_mths, 'E');

    * Calculate the date until which this data is used (3 months after next FYE, end of month) *;
    * The next datadate is approx. 1 year later. 3 months after that is 15 months. *;
    end_avail_date = intnx('MONTH', datadate, 12 + 3, 'E'); /* Approx. 15 months after current FYE */

    * Ensure we only keep data relevant for the sample period calculation *;
    if year(end_avail_date) >= &start_year;

    format avail_date end_avail_date yymmddn8.;
    keep gvkey permno datadate fyear ceq ib dvc avail_date end_avail_date;
run;

* Expand Compustat data to monthly frequency based on availability window *;
data comp_expanded_monthly;
    set comp_monthly_avail;
    do month_date = avail_date to end_avail_date by 1;
        month_date = intnx('MONTH', month_date, 0, 'E'); /* Ensure it's month-end date */
        if year(month_date) >= &start_year and year(month_date) <= &end_year;
           output;
    end;
    format month_date yymmddn8.;
    keep gvkey permno month_date ceq ib dvc fyear datadate;
run;



514  ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
515  
516  
517  * Calculate availability dates for Compustat data *;
518  proc sort data=ccm_annual; by gvkey datadate; run;
NOTE: Input data set is already sorted, no sorting done.
NOTE: PROCEDURE SORT used (Total process time):
      real time           0.00 seconds
      cpu time            0.00 seconds
      
519  
520  data comp_monthly_avail;
521      set ccm_annual;
522      by gvkey;
523  
524      * Calculate the date when the annual data is assumed available (end of month) *;
SYMBOLGEN:  Macro variable REPORT_LAG_MTHS resolves to 4
525      avail_date = intnx('MONTH', datadate, &report_lag_mths, 'E');
526  
527      * Calculate the date until which this data is used (3 months after next FYE, end of month) *;
528      * The next datadate is approx. 1 year later. 3 month

In [10]:
proc print data=comp_expanded_monthly(where=(gvkey="001004" and fyear=2022) obs=20);run;

Obs,gvkey,datadate,fyear,ceq,dvc,ib,month_date
637,001004,20230531,2022,1099.1000,0.0000,89.8000,20230930
638,001004,20230531,2022,1099.1000,0.0000,89.8000,20231031
639,001004,20230531,2022,1099.1000,0.0000,89.8000,20231130
640,001004,20230531,2022,1099.1000,0.0000,89.8000,20231231
641,001004,20230531,2022,1099.1000,0.0000,89.8000,20240131
642,001004,20230531,2022,1099.1000,0.0000,89.8000,20240229
643,001004,20230531,2022,1099.1000,0.0000,89.8000,20240331
644,001004,20230531,2022,1099.1000,0.0000,89.8000,20240430
645,001004,20230531,2022,1099.1000,0.0000,89.8000,20240531
646,001004,20230531,2022,1099.1000,0.0000,89.8000,20240630


### 1.4 CRSP Monthly Return

In [11]:
/* ********************************************************************************* */
/* ******************** W R D S   R E S E A R C H   M A C R O S ******************** */
/* ********************************************************************************* */
/* WRDS Macro: CRSPMERGE                                                             */
/* Summary   : Merges CRSP Stocks and Events Data                                    */
/* Date      : April 14, 2009                                                        */
/* Author    : Rabih Moussawi and Luis Palacios, WRDS                                */
/* https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-crspmerge/*/
/* Variables : - S: Monthly/Daily, defaults to Monthly, but s=d for CRSP Daily data  */
/*             - START, END: Start and End Dates. Example Date Format: 01JAN2000     */
/*             - SFVARS: Stock File Vars to extract. e.g. PRC VOL SHROUT             */
/*             - SEVARS: Event File Vars to extract. e.g. TICKER NCUSIP SHRCD EXCHCD */
/*                  warning: including DIVAMT may result in multiple obs per period  */
/*             - FILTERS: Additional screens using variables in SFVARS or SEVARS     */                  
/*                          (default no filters                                      */
/*             - OUTSET: Output Dataset Name (default names crsp_m or crsp_d)        */
/* ********************************************************************************* */

%MACRO NWORDS (INVAR);

/* WRDS Macro: NWORDS                                                                */
/* Summary   : Counts the Number of Words Within a Text String                       */
/* Date      : January, 2001                                                         */
/* Author    : Mark Keintz, WRDS                                                     */
/* Variables : - INVAR: Input text string                                            */

 
%local N W;
 
/* %let invar = %sysfunc(compbl(&invar)); */
 
%let N = 0;
%let W = 1;
 
%do %while (%nrquote(%scan(&invar,&W,%str( ))) ^= %str());
  %let N = %eval(&N+1);
  %let W = %eval(&W+1);
%end;
 
&N
 
%MEND NWORDS;

 
%MACRO CRSPMERGE (S=m,START=01JAN2000,END=30JUN2001,SFVARS=prc ret shrout,SEVARS=ticker ncusip exchcd shrcd siccd,FILTERS=,OUTSET=crsp_&s.);
 
/* Check Series: Daily or Monthly and define datasets - Default is Monthly  */
%if &s=D %then %let s=d; %else %if &s ne d %then %let s=m;
%if (%sysfunc(libref(crsp))) %then %do;
  %let cs=/wrds/crsp/sasdata/;
  libname crsp ("&cs/m_stock","&cs/q_stock","&cs/a_stock");
%end;
%let sf       = crsp.&s.sf ;
%let se       = crsp.&s.seall ;
%let senames  = crsp.&s.senames ;
 
%put ;
%put #### START. Merging CRSP Stock File (&s.sf) and Event File (&s.se) ;
 
options nonotes;
%let sdate = %sysfunc(putn("&start"d,5.)) ;
%let edate = %sysfunc(putn("&end"d,5.)) ;
 
%let sevars   = %sysfunc(compbl(&sevars));
%let sevars   = %sysfunc(lowcase(&sevars));
%let nsevars  = %nwords(&sevars);
 
/* create lag event variable names to be used in the RETAIN statement */
%let sevars_l = lag_%sysfunc(tranwrd(&sevars,%str( ),%str( lag_)));
 
%if %length(&filters) > 2 %then %let filters = and &filters;
  %else %let filters = %str( );
 
/* Get stock data */
proc sql;
    create table __sfdata
    as select *
    from &sf (keep= permno date &sfvars)
    where date between &sdate and &edate and permno in
    (select distinct permno from
      &senames(WHERE=(&edate>=NAMEDT and &sdate<=NAMEENDT)
         keep=permno namedt nameendt) )
    order by permno, date;
    quit;
 
/* Get event data */
proc sql;
   create table __sedata
   as select a.*
   from &se (keep= permno date &sevars) as a,
    (select distinct permno, min(namedt) as minnamedt from
      &senames(WHERE=(&edate>=NAMEDT and &sdate<=NAMEENDT)
         keep=permno namedt nameendt) group by permno) as b
    where a.date >= b.minnamedt and a.date <= &edate and a.permno =b.permno
   order by a.permno, a.date;
   quit;
 
/* Merge stock and event data */
%let eventvars = ticker comnam ncusip shrout siccd exchcd shrcls shrcd shrflg trtscd nmsind mmcnt nsdinx;
 
data &outset. (keep=permno date &sfvars &sevars);
merge __sedata (in=eventdata) __sfdata (in=stockdata);
by permno date; retain &sevars_l;
%do i = 1 %to &nsevars;
  %let var   = %scan(&sevars,&i,%str( ));
  %let var_l = %scan(&sevars_l,&i,%str( ));
  %if %sysfunc(index(&eventvars,&var))>0 %then
   %do;
     if eventdata or first.permno then &var_l = &var. ;
     else if not eventdata then &var = &var_l. ;
   %end;
 %end;
if eventdata and not stockdata then delete;
drop &sevars_l ;
run;
 
/* Some companies have many distribution on the same date (e.g. a stock and cash dist)  */
/* Records will identical except for different DISTCD and DISTAMT */
proc sort data=&outset. noduplicates;
where 1 &filters;
    by permno date;
run;
 
/* House Cleaning */
proc sql;
drop table __sedata, __sfdata;
quit;
 
options notes;
%put #### DONE . Dataset &outset. Created! ;    %put ;
 
%MEND CRSPMERGE;
 
/* ********************************************************************************* */
/* *************  Material Copyright Wharton Research Data Services  *************** */
/* ****************************** All Rights Reserved ****************************** */
/* ********************************************************************************* */


271  ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
272  
273  /* ********************************************************************************* */
274  /* ******************** W R D S   R E S E A R C H   M A C R O S ******************** */
275  /* ********************************************************************************* */
276  /* WRDS Macro: CRSPMERGE                                                             */
277  /* Summary   : Merges CRSP Stocks and Events Data                                    */
278  /* Date      : April 14, 2009                                                        */
279  /* Author    : Rabih Moussawi and Luis Palacios, WRDS                                */
280  /* https://wrds-www.wharton.upenn.edu/pages/wrds-research/macros/wrds-macro-crspmerge/*/
281  /* Variables : - S: Monthly/Daily,

In [12]:
%let begdate=01JAN1963;
%let enddate=31DEC2024;
 
/* Step 2. Extract CRSP Data for NYSE and AMEX Common Stocks */
/* Merge historical codes with CRSP Monthly Stock File       */
/* Restriction on Share Code: common shares only             */
/*      and Exchange Code: NYSE and AMEX securities only     */
%let filtr = (shrcd in (10,11) and exchcd in (1,2, 3));
/*  Selected variables from the CRSP Monthly Stock File      */
%let fvars =  prc ret shrout cfacpr cfacshr;
/*  Selected variables from the CRSP Monthly Event File      */
%let evars =  shrcd exchcd siccd;
/* Invoke CRSPMERGE WRDS Research Macro. Data Output: CRSP_M */

%crspmerge(s=m,start=&begdate,end=&enddate,sfvars=&fvars,sevars=&evars,filters=&filtr);
 


413  ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
414  
415  %let begdate=01JAN1963;
416  %let enddate=31DEC2024;
417  
418  /* Step 2. Extract CRSP Data for NYSE and AMEX Common Stocks */
419  /* Merge historical codes with CRSP Monthly Stock File       */
420  /* Restriction on Share Code: common shares only             */
421  /*      and Exchange Code: NYSE and AMEX securities only     */
422  %let filtr = (shrcd in (10,11) and exchcd in (1,2, 3));
423  /*  Selected variables from the CRSP Monthly Stock File      */
424  %let fvars =  prc ret shrout cfacpr cfacshr;
425  /*  Selected variables from the CRSP Monthly Event File      */
426  %let evars =  shrcd exchcd siccd;
427  /* Invoke CRSPMERGE WRDS Research Macro. Data Output: CRSP_M */
428  
429  %crspmerge(s=m,start=&begdate,end=&enddate,sfvars=&fvars,sevars=&evars,filte

In [22]:
/* Step 4: Add your custom calculations (market_cap, month_date) */
data crsp_monthly0;
    set crsp_m;
    /* Calculate Market Cap (shrout is in 1000s) */
    if prc ne . and shrout ne . and shrout > 0 then do;
        price = abs(prc); /* Use absolute price (CRSP uses negative for bid/ask avg) */
        market_cap = price * shrout * 1000;
    end;
    else do;
        price = .;
        market_cap = .;
    end;
    /* Rename date to match Compustat monthly date */
    month_date = intnx('MONTH', date, 0, 'E'); /* Ensure month-end */
    format month_date yymmddn8.;
    /* Keep your desired variables */
    keep permno permco date month_date price shrout market_cap ret;
run;


* Optional: Check the output;
proc print data=crsp_monthly0(obs=10);
    title "First 10 Observations of crsp_monthly";
run;

Obs,DATE,PERMNO,RET,SHROUT,price,market_cap,month_date
1,19860131,10000,C,3680,4.37500,16100000.00,19860131
2,19860228,10000,-0.257143,3680,3.25000,11960000.00,19860228
3,19860331,10000,0.365385,3680,4.43750,16330000.00,19860331
4,19860430,10000,-0.098592,3793,4.00000,15172000.00,19860430
5,19860530,10000,-0.222656,3793,3.10938,11793859.38,19860531
6,19860630,10000,-0.005025,3793,3.09375,11734593.75,19860630
7,19860731,10000,-0.080808,3793,2.84375,10786343.75,19860731
8,19860829,10000,-0.615385,3793,1.09375,4148593.75,19860831
9,19860930,10000,-0.057143,3793,1.03125,3911531.25,19860930
10,19861031,10000,-0.242424,3843,0.78125,3002343.75,19861031


In [29]:
proc sort data=crsp.cs5yr out=treasury(rename=(qtdate=date)); by qtdate; run;
proc sort data=crsp_monthly0; by date; run;

data crsp_monthly; merge crsp_monthly0(in=_c) treasury(in=_f keep=date yldmat); by date; if _c and _f; run;

proc print data=crsp_monthly(obs=10);
    title "First 10 Observations of crsp_monthly";
run;



Obs,DATE,PERMNO,RET,SHROUT,price,market_cap,month_date,YLDMAT
1,19630131,10006,0.047002,1453,80.750,117329750,19630131,3.664
2,19630131,10014,0.034483,3294,3.750,12352500,19630131,3.664
3,19630131,10030,0.056122,1626,51.750,84145500,19630131,3.664
4,19630131,10057,0.057471,500,57.000,28500000,19630131,3.664
5,19630131,10102,0.041475,5013,56.500,283234500,19630131,3.664
6,19630131,10137,0.053846,9338,51.375,479739750,19630131,3.664
7,19630131,10145,0.025424,26466,45.375,1200894750,19630131,3.664
8,19630131,10153,0.162393,8956,17.000,152252000,19630131,3.664
9,19630131,10161,0.125000,14295,34.875,498538125,19630131,3.664
10,19630131,10188,0.157407,1867,15.625,29171875,19630131,3.664


### 1.5 Merge CRSP/Compustat Linked data with Expanded Compustat  

In [32]:
proc sort data=crsp_monthly;
    by permno month_date;
run;


proc sort data=comp_expanded_monthly;
    by permno month_date;
run;


data firm_monthly_data;
    merge crsp_monthly(in=a) comp_expanded_monthly(in=b);
    by permno month_date;
    if a and b; /* Keep only firms with both CRSP and lagged Compustat data */

    * Keep relevant variables for aggregation and Vf calculation *;
    keep gvkey datadate fyear permno permco month_date market_cap ceq ib dvc ret yldmat;
run;



785  ods listing close;ods html5 (id=saspy_internal) file=stdout options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: STDOUT
786  
787  proc sort data=crsp_monthly;
788      by permno month_date;
789  run;
NOTE: Input data set is already sorted, no sorting done.
NOTE: PROCEDURE SORT used (Total process time):
      real time           0.00 seconds
      cpu time            0.00 seconds
      
790  
791  
792  proc sort data=comp_expanded_monthly;
793      by permno month_date;
794  run;
NOTE: Input data set is already sorted, no sorting done.
NOTE: PROCEDURE SORT used (Total process time):
      real time           0.00 seconds
      cpu time            0.00 seconds
      
795  
796  
797  data firm_monthly_data;
798      merge crsp_monthly(in=a) comp_expanded_monthly(in=b);
799      by permno month_date;
800      if a and b; /* Keep only firms with both CRSP and lagged Compustat data */
801  
802     

In [31]:
proc means data=firm_monthly_data; run;

Variable,Label,N,Mean,Std Dev,Minimum,Maximum
PERMNO RET market_cap month_date YLDMAT ceq dvc ib,PERMNO Returns Annualized Yield-to-Maturity Common/Ordinary Equity - Total Dividends Common/Ordinary Income Before Extraordinary Items,2515112 2495951 2496767 2515112 2515112 2472062 2464201 2468599,56730.35 0.0117336 2993207560 14876.27 5.4215942 995.9340641 50.1690939 122.3103066,28407.73 0.1946009 26455977194 4516.55 3.2624449 6781.50 408.1344531 1256.93,10000.00 -0.9936000 27703.13 6970.00 0.2050000 -96620.00 -10.0000000 -99289.00,93436.00 26.5838280 3.7853044E12 23741.00 15.5470000 561273.00 36968.00 99803.00


### 1.6 Calculate Firm-Level Fundamental Value (Vf(x))

In [37]:
data firm_monthly_vf;
    set firm_monthly_data;

    * Calculate discount rate (r_t) - annual rate  %let equity_premium = 0.06   6% equity premium ;
    r_t = (YLDMAT / 100) + &equity_premium;

    * Calculate residual income (RI) using f(1)=lagged IB *;
    ri = ib - r_t * ceq;

    * Calculate Vf(x) using Equation 11, growth rate is 3% aka 0.03 *;
    if r_t > &growth_rate then do;
       vf_x = ceq + ri / (r_t - &growth_rate);
    end;
    else do;
       * Handle case where r <= g, perpetuity formula invalid. *;
       * Apply floor or alternative. Paper isn't explicit, flooring at CEQ seems plausible.*;
       vf_x = ceq;
    end;

    * Apply floor based on paper's description: *;
    * "Observations where residual income is negative... are retained, but *;
    *  with the residual income model estimate set equal to book-value." *;
    if ri <= 0 then vf_x = ceq;

    * Ensure Vf(x) is not missing if components are available *;
     if vf_x = . and ceq ne . and ib ne . and r_t ne . then vf_x = ceq; /* Fallback */

    if vf_x>0 then lg_vf_x = log(vf_x); 
    keep gvkey datadate fyear month_date yldmat market_cap ceq dvc ib ri vf_x lg_vf_x;
run;

proc means data=firm_monthly_vf n mean min p1 p5 p25 p50 p75; var ceq yldmat ri vf_x lg_vf_x; run;

Variable,Label,N,Mean,Minimum,1st Pctl,5th Pctl,25th Pctl,50th Pctl,75th Pctl
ceq YLDMAT ri vf_x lg_vf_x,Common/Ordinary Equity - Total Annualized Yield-to-Maturity,2472062 2515112 2468149 2472062 2413161,995.9340641 5.4215942 30.0325878 2378.83 4.7430723,-96620.00 0.2050000 -103985.79 -96620.00 -6.9077553,-111.7840000 0.3190000 -632.8171008 -23.3560000 -0.2810375,1.0510000 0.8470000 -122.7282103 1.5990000 1.0021014,15.8760000 2.8070000 -11.2183742 19.2480000 3.0872938,74.0060000 5.1340000 -1.2572447 93.9528579 4.6162804,356.0850000 7.4770000 2.8745683 524.3340000 6.3120977
